In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score , mean_absolute_error

from catboost import CatBoostRegressor as cbr

In [ ]:
from google.colab.drive import mount
mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/My Drive/files/House_Price_dataset.csv')
data.head()

In [ ]:
data.drop(['property_id' , 'location_id' , 'page_url' , 'agency' , 'agent' , 'Area Category' , 'date_added'] , axis=1 , inplace=True)

In [ ]:
data['lastPrice'] = data['price']
data.drop(['price'] , axis=1 , inplace=True)

In [ ]:
x = data.iloc[: , :-1]
y = data.iloc[: , -1]

In [ ]:
le = LabelEncoder()
x.iloc[: , 0] = le.fit_transform(x.iloc[: , 0])
x.iloc[: , 1] = le.fit_transform(x.iloc[: , 1])
x.iloc[: , 2] = le.fit_transform(x.iloc[: , 2])
x.iloc[: , 3] = le.fit_transform(x.iloc[: , 3])
x.iloc[: , 7] = le.fit_transform(x.iloc[: , 7])
x.iloc[: , 8] = le.fit_transform(x.iloc[: , 8])
x.iloc[: , 10] = le.fit_transform(x.iloc[: , 10])

In [ ]:
scaler = StandardScaler()
minmax = MinMaxScaler(clip=True)
x = scaler.fit_transform(x)
x = minmax.fit_transform(x)

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , random_state=1441)

In [ ]:
model = cbr(random_state=1441 , learning_rate=0.001 ,use_best_model=True , verbose=6000 , task_type='GPU' , iterations=30000 , max_depth=10 , model_size_reg=10000)

In [ ]:
model.fit(x_train , y_train , eval_set=(x_test , y_test))

In [ ]:
pred = model.predict(x_test)
print(r2_score(y_test , pred))
print(mean_absolute_error(y_test , pred))

In [ ]:
model.save_model('/content/drive/My Drive/models/House/Model.h5')